# The "I can't vote" Emergency (Ticket Management)

In [ ]:
from main import app, db, TicketMeta, PollTable, StudyMeta, CriteriaMeta, AdminControls
from modules.qr_code import generate_tokens
import uuid
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
import qrcode
import os
import pandas as pd
from tqdm import tqdm


# DIAGNOSTIC: Check why a ticket isn't working
def inspect_ticket(ticket_id):
    with app.app_context():
        ticket = TicketMeta.query.get(ticket_id)
        if not ticket:
            print(f"❌ Ticket {ticket_id} NOT FOUND.")
            return
        
        # Check if they actually voted in the PollTable
        votes = PollTable.query.filter_by(ticket_id=ticket_id).all()
        vote_count = len(votes)
        
        print(f"--- Ticket Inspector: {ticket_id} ---")
        print(f"Status in TicketMeta: {'VALID' if ticket.ticket_valid else 'USED/INVALID'}")
        print(f"Actual Votes Cast: {vote_count}")
        
        if vote_count > 0:
            print("Votes found for:")
            for v in votes:
                study = StudyMeta.query.get(v.study_id)
                print(f" - {study.study_name}")
        else:
            print("No votes recorded in PollTable.")

# FIX: Reset ONE specific ticket (Surgical fix)
def reset_single_ticket(ticket_id):
    with app.app_context():
        try:
            # 1. Reset Ticket Status
            ticket = TicketMeta.query.get(ticket_id)
            if ticket:
                ticket.ticket_valid = True
                
                # 2. Delete any previous votes associated with this ticket
                # This prevents double counting if they vote again
                deleted = db.session.query(PollTable).filter_by(ticket_id=ticket_id).delete()
                
                db.session.commit()
                print(f"✅ Ticket {ticket_id} reset. {deleted} previous votes deleted. User can vote again.")
            else:
                print(f"❌ Ticket {ticket_id} not found.")
        except Exception as e:
            db.session.rollback()
            print(f"Error: {e}")

# EMERGENCY: Create new tickets on the fly
def generate_emergency_tickets(count=10, pdf_out_path):
    with app.app_context():
        new_tickets = generate_tokens(count)
        for idx in range(count):
            t = TicketMeta(ticket_id=new_tickets[idx], ticket_valid=True)
            db.session.add(t)

        
        db.session.commit()
        print(f"✅ Created {count} new tickets!")
        print("Here are the codes (Copy these):")
        for t in new_tickets:
            print(t)
        
        # Add Emergency Tickets to final_tokens.csv
        old_tokens = list(pd.read_csv(r'seed_db\final_tokens.csv')['Tokens'])
        all_tokens = old_tokens + new_tickets
        df = pd.DataFrame({'Tokens': all_tokens})
        df.to_csv(os.path.join(os.getcwd(),
                           'seed_db',
                           'final_tokens.csv'),
                           index=False)


        #Make PDF of emergency tickets
        c = canvas.Canvas(pdf_out_path, pagesize=A4)
        width, height = A4
        
        # 2. Define Grid Settings (A4 is 210mm x 297mm)
        margin_x = 10 * mm
        margin_y = 10 * mm
        
        cols = 5
        rows = 10
        
        # Calculate cell size
        cell_width = (width - (2 * margin_x)) / cols 
        cell_height = (height - (2 * margin_y)) / rows
        
        # QR Code Image Size (slightly smaller than cell to add padding)
        qr_size = 25 * mm 

        # 3. Fetch all tickets from Database

        tickets = new_tickets
        print(f"Generating PDF for {len(tickets)} tickets...")

        current_col = 0
        current_row = 0
        i = 0
        for ticket in tqdm(tickets):
            # --- A. Generate QR Code Image ---
            qr = qrcode.QRCode(box_size=10, border=1)
            qr_data = f"http://xucoefypdp26peoplechoice-production.up.railway.app/vote/{ticket}"
            qr.add_data(qr_data)
            qr.make(fit=True)
            
            img = qr.make_image(fill_color="black", back_color="white")
            
            # Save temporarily to draw it (ReportLab needs a file path)
            temp_path = f"temp_qr_{i}.png"
            temp_path = os.path.join(r"qr_codes\ind_qr", temp_path)
            img.save(temp_path)

            # --- B. Calculate Coordinates ---
            x_pos = margin_x + (current_col * cell_width)
            y_pos = height - margin_y - ((current_row + 1) * cell_height)
            
            # Center the QR inside the cell
            x_centered = x_pos + (cell_width - qr_size) / 2
            y_centered = y_pos + (cell_height - qr_size) / 2

            # --- C. Draw on PDF ---
            c.drawImage(temp_path, x_centered, y_centered, width=qr_size, height=qr_size)
            
            # Optional: Write the Ticket ID text below the QR
            c.setFont("Helvetica", 5)
            c.drawCentredString(x_centered + qr_size/2, y_centered - 2*mm, ticket)

            # Clean up temp file
            os.remove(temp_path)

            # --- D. Move to Next Grid Slot ---
            current_col += 1
            if current_col >= cols:
                current_col = 0
                current_row += 1
                
            # --- E. Check if Page is Full ---
            if current_row >= rows:
                c.showPage() # Create new page
                current_col = 0
                current_row = 0
            
            #Update
            i+=1

        # 4. Save
        c.save()
        print(f"Success! Saved to {pdf_out_path}")

# The "Typos & Content" Emergency

In [ ]:
# FIX: Rename a study
def fix_typo_study(study_id, new_name):
    with app.app_context():
        study = StudyMeta.query.get(study_id)
        if study:
            old_name = study.study_name
            study.study_name = new_name
            db.session.commit()
            print(f"✅ Renamed '{old_name}' to '{new_name}'")
        else:
            print(f"❌ Study ID {study_id} not found.")

# FIX: Add a missing study
def add_missing_study(category_id, study_name):
    with app.app_context():
        # 1. Add to StudyMeta
        new_study = StudyMeta(study_name=study_name)
        db.session.add(new_study)
        db.session.commit() # Commit to get the study_id
        
        # 2. Link to Category (AllStudies)
        # Note: Depending on your model, this might be 'AllStudies' or a direct relationship
        from main import AllStudies 
        link = AllStudies(criteria_id=category_id, study_id=new_study.study_id)
        db.session.add(link)
        db.session.commit()
        
        print(f"✅ Added '{study_name}' \n(ID: {new_study.study_id}) to Category {category_id}")

# The "Data Integrity" Emergency

In [ ]:
# AUDIT: Check for data inconsistencies
def audit_database():
    with app.app_context():
        print("--- STARTING AUDIT ---")
        
        # Check 1: Tickets marked USED but have NO VOTES (Failed transactions?)
        invalid_tickets = TicketMeta.query.filter_by(ticket_valid=False).all()
        suspicious_count = 0
        
        for t in invalid_tickets:
            vote_count = PollTable.query.filter_by(ticket_id=t.ticket_id).count()
            if vote_count == 0:
                print(f"⚠️  WARNING: Ticket {t.ticket_id} is marked USED, but has 0 votes.")
                suspicious_count += 1
                
        if suspicious_count == 0:
            print("✅ Ticket Integrity: OK")
        else:
            print(f"❌ Found {suspicious_count} suspicious tickets.")

        # Check 2: Votes with NO Valid Ticket (Orphaned votes)
        # This requires a JOIN or a loop depending on DB size
        all_votes = PollTable.query.all()
        orphans = 0
        for v in all_votes:
            if not TicketMeta.query.get(v.ticket_id):
                print(f"⚠️  ORPHAN VOTE: Vote ID {v.poll_id} has ticket {v.ticket_id} which does not exist.")
                orphans += 1
        
        if orphans == 0:
            print("✅ Vote Integrity: OK")

# BACKUP: Export everything to CSV immediately
def backup_data_to_csv():
    with app.app_context():
        # Use pandas for quick export
        votes_df = pd.read_sql(db.session.query(PollTable).statement, db.session.bind)
        votes_df.to_csv('backup_votes.csv', index=False)
        
        tickets_df = pd.read_sql(db.session.query(TicketMeta).statement, db.session.bind)
        tickets_df.to_csv('backup_tickets.csv', index=False)
        
        print("✅ Database backed up to 'backup_votes.csv' and 'backup_tickets.csv'")

# Nuclear Option

In [ ]:
def nuke_and_reset():
    confirmation = input("🚨 DANGER: This will DELETE ALL VOTES. Type 'DELETE' to confirm: ")
    if confirmation != "DELETE":
        print("Operation cancelled.")
        return

    with app.app_context():
        try:
            # 1. Delete Votes
            deleted_rows = db.session.query(PollTable).delete()
            
            # 2. Reset Tickets
            rows = db.session.query(TicketMeta).update({TicketMeta.ticket_valid: True})
            
            db.session.commit()
            print(f"☢️  NUCLEAR RESET COMPLETE.")
            print(f" - {deleted_rows} votes deleted.")
            print(f" - {rows} tickets re-activated.")
            
        except Exception as e:
            db.session.rollback()
            print(f"Error: {e}")